In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [7]:
#https://discuss.pytorch.org/t/how-to-reverse-a-torch-tensor/382
def reverse(x):
    #reverse first dimension
    #first - create a tensor with reversed dims 
    idx = [i for i in range(x.size(0)-1, -1, -1)]
    print(idx)
    idx = torch.LongTensor(idx)

    #create tensor with reversed dims
    inverted_tensor = x.index_select(0, idx)

    return inverted_tensor


In [8]:
x = Variable(torch.randn(10,2,2))

In [9]:
print(x)

tensor([[[ 1.8419, -1.1705],
         [-0.8331,  1.1257]],

        [[-0.6159,  0.6571],
         [-0.4614,  0.0858]],

        [[ 1.2304, -0.0010],
         [-0.4309, -0.5960]],

        [[ 0.4960,  0.0308],
         [-1.3932, -0.2724]],

        [[ 2.1710,  2.2022],
         [ 0.2743, -0.9559]],

        [[-0.6489,  0.1208],
         [-0.1102, -0.5246]],

        [[-1.9245, -0.4373],
         [ 0.8161,  1.1235]],

        [[ 0.3120, -0.7495],
         [-1.2842, -2.9873]],

        [[-0.9139, -0.1609],
         [-0.0004,  0.3704]],

        [[ 0.1085, -1.4659],
         [ 0.2962,  1.2593]]])


In [10]:
reverse(x)

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


tensor([[[ 0.1085, -1.4659],
         [ 0.2962,  1.2593]],

        [[-0.9139, -0.1609],
         [-0.0004,  0.3704]],

        [[ 0.3120, -0.7495],
         [-1.2842, -2.9873]],

        [[-1.9245, -0.4373],
         [ 0.8161,  1.1235]],

        [[-0.6489,  0.1208],
         [-0.1102, -0.5246]],

        [[ 2.1710,  2.2022],
         [ 0.2743, -0.9559]],

        [[ 0.4960,  0.0308],
         [-1.3932, -0.2724]],

        [[ 1.2304, -0.0010],
         [-0.4309, -0.5960]],

        [[-0.6159,  0.6571],
         [-0.4614,  0.0858]],

        [[ 1.8419, -1.1705],
         [-0.8331,  1.1257]]])

In [11]:
class biGRU(nn.Module):
    def __init__(self,input_size, hidden_size):
        super(biGRU, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        #hand designed, so bidirectional is set to False
        self.GRU_f = nn.GRU(input_size, hidden_size, bidirectional=False)
        self.GRU_b = nn.GRU(input_size, hidden_size, bidirectional=False)

    def forward(self, x):
        #make sure that first dimension is timesteps
        batch_size = x.size(1)
        x_f = x
        x_b = reverse(x)

        h_f = Variable(torch.FloatTensor(1, batch_size, self.hidden_size).zero_())
        h_b = torch.zeros_like(h_f)

        #y_f: (T, B, dim)
        #y_b: (T, B, dim)
        y_f, h_f = self.GRU_f(x_f, h_f)
        y_b, h_b = self.GRU_b(x_b, h_b)

        #need to reverse again before concatenating
        y_b = reverse(y_b)
        
        y = torch.cat((y_f, y_b), 2)

        return y
        

In [12]:
gru = biGRU(20,4)

In [13]:
print(gru)

biGRU(
  (GRU_f): GRU(20, 4)
  (GRU_b): GRU(20, 4)
)


In [14]:
x = torch.randn(10,3,20)

In [15]:
y = gru(x)

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


In [16]:
y.size()

torch.Size([10, 3, 8])

In [17]:
x.size()

torch.Size([10, 3, 20])